# Setup

In [1]:
from google.cloud import asset_v1, securitycenter_v1

In [2]:
scc_client = securitycenter_v1.SecurityCenterClient()
cai_client = asset_v1.AssetServiceClient()

# Functions

In [3]:
def get_scc_asset(resource_name):
    maybe_asset = list(
        scc_client.list_assets(
            securitycenter_v1.ListAssetsRequest(
                parent="organizations/129624834409",
                filter=f'security_center_properties.resource_name = "{resource_name}"',
            )
        )
    )
    if len(maybe_asset) == 1:
        return maybe_asset[0]

In [4]:
def get_cai_asset_name(scc_asset):
    scc_properties = scc_asset.asset.security_center_properties
    if scc_properties.resource_type == "google.compute.Instance":
        return "/".join(
            [
                scc_properties.resource_name.rsplit("/", 1)[0],
                scc_properties.resource_display_name,
            ]
        )
    return scc_properties.resource_name

In [5]:
def get_cai_asset(cai_asset_name):
    maybe_asset = list(
        cai_client.search_all_resources(
            asset_v1.SearchAllResourcesRequest(
                scope="organizations/129624834409",
                query=f'name="{cai_asset_name}"',
                page_size=1000,
            )
        )
    )
    if len(maybe_asset) == 1:
        return maybe_asset[0]

# Get all findings associated with instances

In [6]:
scc_instance_findings = list(
    scc_client.list_findings(
        securitycenter_v1.ListFindingsRequest(
            parent="organizations/129624834409/sources/-",
            filter='state="ACTIVE" AND resource_name:"/instances/"',
            page_size=1000,
        )
    )
)

# Pick an instance finding at random

In [12]:
finding = scc_instance_findings[100]
finding.resource

name: "//compute.googleapis.com/projects/projectinfolder-259022/zones/us-central1-a/instances/1794584243532877577"
project_name: "//cloudresourcemanager.googleapis.com/projects/927112907303"
project_display_name: "projectinfolder-259022"
parent_name: "//cloudresourcemanager.googleapis.com/projects/927112907303"
parent_display_name: "projectinfolder-259022"

# Get the associated SCC asset

In [16]:
scc_asset = get_scc_asset(finding.resource.name)
scc_asset.asset.security_center_properties

resource_name: "//compute.googleapis.com/projects/projectinfolder-259022/zones/us-central1-a/instances/1794584243532877577"
resource_type: "google.compute.Instance"
resource_parent: "//cloudresourcemanager.googleapis.com/projects/927112907303"
resource_project: "//cloudresourcemanager.googleapis.com/projects/927112907303"
resource_owners: "user:fwparker@google.com"
resource_display_name: "maybe-public"
resource_parent_display_name: "projectinfolder-259022"
resource_project_display_name: "projectinfolder-259022"

# Construct the CAI asset name using the SCC asset resource properties

In [18]:
cai_asset_name = get_cai_asset_name(scc_asset)
cai_asset_name

'//compute.googleapis.com/projects/projectinfolder-259022/zones/us-central1-a/instances/maybe-public'

# Retrieve the corresponding CAI asset

In [19]:
cai_asset = get_cai_asset(cai_asset_name)

In [20]:
cai_asset

name: "//compute.googleapis.com/projects/projectinfolder-259022/zones/us-central1-a/instances/maybe-public"
asset_type: "compute.googleapis.com/Instance"
project: "projects/927112907303"
display_name: "maybe-public"
location: "us-central1-a"
additional_attributes {
  fields {
    key: "networkInterfaces"
    value {
      list_value {
        values {
          struct_value {
            fields {
              key: "network"
              value {
                string_value: "https://www.googleapis.com/compute/v1/projects/projectinfolder-259022/global/networks/default"
              }
            }
            fields {
              key: "networkIP"
              value {
                string_value: "10.128.0.2"
              }
            }
          }
        }
      }
    }
  }
}